# Create a Graph from a text

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
- Define an Ontology
- Load a list of example text chunks. We will use the Lord of the Rings summary from this wikipedia page. 
- Create Graph using an Open source model using Groq APIs. 
- Save the graph to Neo4j db
- Visualise



Loading the graph maker functions ->

In [12]:
from graph_maker import GraphMaker, Ontology, GroqClient
from graph_maker import Document


# Define the Ontology. 

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```

Here is the ontology we will use for the LOTR summaries ->


In [13]:

ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)


List of text chunks. These are the summaries of the LOTR books from the Wikipedia page. 

In [14]:
from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list
len(example_text_list)

15

## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [15]:


model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"


## Create documents out of text chumks. 
Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 


In [16]:
import datetime
current_time = str(datetime.datetime.now())
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)


## Create Graph
Finally run the Graph Maker to generate graph. 

In [17]:

graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=50 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  mixtral-8x7b-32768


...

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-04 19:11:48 - INFO 
LLM Response:
[
{
"node_1": {"label": "Person", "name": "Sam"},
"node_2": {"label": "Person", "name": "Frodo"},
"relationship": "Sam rescues Frodo from the tower of Cirith Ungol."
},
{
"node_1": {"label": "Person", "name": "Frodo"},
"node_2": {"label": "Place", "name": "tower of Cirith Ungol"},
"relationship": "Frodo was rescued by Sam from the tower of Cirith Ungol."
},
{
"node_1": {"label": "Person", "name": "Sam"},
"node_2": {"label": "Person", "name": "Frodo"},
"relationship": "Sam and Frodo set out across Mordor together."
},
{
"node_1": {"label": "Person", "name": "Frodo"},
"node_2": {"label": "Place", "name": "Cracks of Doom"},
"relationship": "When Frodo and Sam reach the edge of the Cracks of Doom, Frodo cannot resist the Ring any longer."
},
{
"node_1": {"label": "Person", "name": "Frodo"},
"node_2": {"label": "Object", "name": 

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-05-04 19:12:40 - INFO 
LLM Response:
[
{
"node_1": {"label": "Person", "name": "Galadriel"},
"node_2": {"label": "Person", "name": "Treebeard"},
"relationship": "say farewell to"
},
{
"node_1": {"label": "Person", "name": "Celeborn"},
"node_2": {"label": "Person", "name": "Treebeard"},
"relationship": "say farewell to"
},
{
"node_1": {"label": "Person", "name": "Gandalf"},
"node_2": {"label": "Person", "name": "Treebeard"},
"relationship": "say farewell to"
},
{
"node_1": {"label": "Person", "name": "Aragorn"},
"node_2": {"label": "Person", "name": "Treebeard"},
"relationship": "is said farewell to"
},
{
"node_1": {"label": "Person", "name": "Four hobbits"},
"node_2": {"label": "Place", "name": "Home"},
"relationship": "travel to"
},
{
"node_1": {"label": "Place", "name": "Home"},
"node_2": {"label": "Person", "name": "Saruman's men"},
"relationship": "is taken

Total number of Edges 200


In [18]:
for edge in graph:
    print(edge.model_dump(), "\n\n")

{'node_1': {'label': 'Person', 'name': 'Bilbo Baggins'}, 'node_2': {'label': 'Event', 'name': 'birthday celebration'}, 'relationship': 'Bilbo Baggins hosts the birthday celebration', 'metadata': {'summary': "Bilbo Baggins gives his inherited Ring to Frodo, which Gandalf later confirms as a lost Ring of Power from the Dark Lord Sauron. Gandalf urges Frodo to leave the Shire with it, but after seventeen years, he hasn't returned. Frodo embarks on a journey with his cousin Pippin Took and gardener Sam Gamgee, pursued by Black Riders. They encounter Elves who use singing to protect them, then take a shortcut to Bucklebury Ferry, where they meet Merry Brandybuck. Merry and Pippin, aware of the Ring's significance, insist on joining Frodo's journey.", 'generated_at': '2024-05-04 18:59:00.570860'}, 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Bilbo Baggins'}, 'node_2': {'label': 'Object', 'name': 'Ring'}, 'relationship': 'Bilbo Baggins leaves the Ring to Frodo', 'metadata': {'summar

# Save the Graph to Neo4j 

In [19]:
from graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()


aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

200